In [ ]:
from functions import get_current_location
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [1]:
from route import Route
from api_route import ApiRoute

current_position = {'longitude': 130.843618,
                    'latitude': -12.457114}
final_position = {'longitude': 133.8750197,
                  'latitude': -23.7090616}
delta_spacing = 2000.0 # in meters
number_sites = 10

route = Route()
api_route = ApiRoute(route)

# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites)
# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
api_route_df = api_route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites, print_is_requested=False)
# route.find_closest_row(current_position)
# route.find_closest_rows(route.get_control_stops_data)


In [ ]:
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser

api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser, print_is_requested=True)
api_executer = ApiExecuter(requester=api_requester)

In [ ]:
# api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [ ]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(api_route_df, print_is_requested=True)

In [ ]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [ ]:
# site_id = 54689
# name = "404"
# position = {'latitude': 136,
#             'longitude': -30}

# api_requester.get_site_edit(site_id, name=name, position=position)

api_route_df.index=api_executer.get_all_site_id ########TODO ADAPT DIRECTLY 
api_executer.edit_sites(api_route_df)

In [ ]:
# raw_forecast_df = api_requester.get_solar_forecast()
# raw_forecast_df, forecast_ready = api_executer.get_new_solar_forecast()

raw_forecast_df = api_requester.get_solar_forecast_cloudmove()
# raw_forecast_df, forecast_ready = api_executer.get_new_solar_forecast_cloudmove()

print(raw_forecast_df)

In [ ]:
sites_df = api_requester.get_current_sites
print(sites_df)

from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = None
forecast_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_forecast_df, hours_in_advance)
preprocessor.save_data2folder()
print(preprocessed_df)

In [ ]:
import folium

# Assuming you have these dataframes already
control_stops_df = route.get_control_stops_data
sites_df = api_requester.get_current_sites
rt = route.get_route_data

# Define the bounds
min_lng = rt['longitude'].min()
min_lat = rt['latitude'].min()
max_lng = rt['longitude'].max()
max_lat = rt['latitude'].max()
bounds = [[max_lat, min_lng], [min_lat, max_lng]]

# Create a base map centered around Australia
middle_lat = (max_lat - min_lat) / 2
middle_lng = (max_lng - min_lng) / 2
map = folium.Map(location=[middle_lat, middle_lng],
                 zoom_start=0)

# Set the map boundaries to show only Australia
map.fit_bounds(bounds)

# Add the route lines
folium.PolyLine(
    locations=rt[['latitude', 'longitude']].values.tolist(),
    color="green",
    weight=4,
    opacity=1
).add_to(map)

# Add api sites
for _, row in sites_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=1,
        tooltip=row['name']
    ).add_to(map)

# Add control stops
for _, row in control_stops_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color="orange",
        fill=True,
        fill_color="orange",
        fill_opacity=1,
        tooltip=row['town'] + ': ' + row['location']
    ).add_to(map)

folium.Marker(
                location=[current_position['latitude'], current_position['longitude']],
                icon=folium.Icon(icon="car", prefix="fa")
        ).add_to(map)

map.options['maxBounds'] = bounds
map.options['maxBoundsViscosity'] = 0.9
map.options['minZoom'] = 5

def add_legend(map_obj):
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; width: 120px; height: 90px; 
                border:2px solid grey; z-index:9999; font-size:14px;">
    &nbsp;<b>Legend</b><br>
    &nbsp;<i class="fa-circle fa-2x" style="color:green"></i>&nbsp;Route<br>
    &nbsp;<i class="fa-circle fa-2x" style="color:blue"></i>&nbsp;API Sites<br>
    &nbsp;<i class="fa-circle fa-2x" style="color:orange"></i>&nbsp;Control Stops
    </div>
    """
    legend = folium.Html(legend_html, script=True)
    folium.Popup(legend).add_to(map_obj)

# Add the legend to the map
add_legend(map)

map


In [2]:
from plotter import Plotter

plotter = Plotter(route)
# plotter.add_api_sites(api_requester.get_current_sites)
plotter.add_control_stops(route.get_control_stops_data)
# plotter.add_current_position(current_position)
current_position = {'longitude': 138.606376,
                    'latitude': -34.932661}
plotter.update_max_speed_distance(current_position)
plotter.plot

Nearest index in csv file: 11929
Nearest index in dataframe: 11927
        longitude   latitude  altitude  altitudeSmoothed  inclination  \
11927  138.606376 -34.932661     56.75         55.592130     0.000000   
11928  138.606381 -34.932727     56.75         55.621667    -0.022426   
11929  138.606423 -34.933227     55.50         55.843854    -0.026249   
11930  138.606367 -34.933392     55.00         55.923660    -0.021112   
11931  138.606414 -34.933923     53.75         56.179127    -0.027017   
11932  138.606421 -34.934006     53.50         56.219099    -0.030327   
11933  138.606426 -34.934080     53.25         56.254572     0.060379   
11934  138.606464 -34.934637     57.00         56.524426     0.035668   
11935  138.606509 -34.935266     59.50         56.766948    -0.020979   
11936  138.606516 -34.935373     59.25         56.799025     0.071052   
11937  138.606663 -34.935335     60.25         56.833683    -0.053200   
11938  138.606712 -34.935322     60.00         56.844459 

KeyError: 0